In [23]:
#Run this preamble to import some libraries that are available in google colab that are often useful.
#Numpy is good for efficiently working with array/vector/matrix data.
#Random is good for generating random numbers according to some (discrete or continuous) distribution
#Matplotlib is good for plotting
#Torch is PyTorch, which is the standard python library for creating and training ML models
#You may need to call other libraries for your code

import numpy as np
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from operator import neg

## Importing SubroutineA from Non Visibly Reduced Notebook:

In [24]:
#note: copied the function from CelinaRoman's notebook
def reduce_coxeter_word(w):
  reduced = True
  while reduced:
    reduced = False
    i = 0
    while i < len(w) - 1:
      if w[i] == w[i+1]: #Removes any s^2
        w = w[:i] + w[i+2:]
        i = max(i-1,0)
      else:
        i += 1
  return w

def reduce_artin_word(w):
  stack = []
  for x in w:
    if stack and stack[-1] == -x:
      stack.pop()
    else:
      stack.append(x)
  return stack

# Coxeter Matrix Check

 Given user input: Rank of Coxeter matrix (= Number of vertices of Coxeter graph = Number of rows, columns of Coxeter matrix) and numpy array encoding rows of Coxeter matrix, output either “This is a valid Coxeter matrix” or “This is not a valid Coxeter matrix. A Coxeter matrix has the following properties: *********”)


In [25]:
def is_coxeter_matrix(n, m):

    # Converts values to numeric, fails if any non-numeric values
    try:
        m = m.astype(float)
    except ValueError:
        print("Non-numeric error: Matrix contains non-numeric entries.")
        return


    # Checks if input matrix is of size rank x rank
    if m.shape != (n, n):
        print(f"Invalid shape: Expected a square matrix of size {n}×{n}, but got {m.shape}.")
        return

    # Checks if diagonal entries are 1
    if not np.all(np.diag(m) == 1):
        print("Invalid diagonal: All diagonal entries must be 1 in a Coxeter matrix.")

    # Check if matrix is symmetric (including handling NaN/inf)
    if not np.allclose(m, m.T, equal_nan=True):
        print("Symmetry error: Coxeter matrices must be symmetric across the diagonal.")
        return

    # Mask off diagonal ones if they exist
    off_diagonal = ~np.eye(n, dtype=bool)
    invalid_ones = (m == 1) & off_diagonal

    # Checks for off diagonal ones
    if np.any(invalid_ones):
        print("Off-diagonal 1s detected: Only diagonal entries can be 1 in a Coxeter matrix.")
        return

    # Mask infinities before checking cast
    is_inf = np.isinf(m)
    is_pos_int = (m > 0) & ~is_inf & (m == np.floor(m))
    valid_entries = is_inf | is_pos_int

    # Checks if all values are positive, and if they are an integer or an infinity
    if not np.all(valid_entries):
        print("Entry error: All off-diagonal entries must be integers ≥ 2 or ∞.")
        return

    print("This is a valid Coxeter matrix.")


# Subroutine B

Subroutine B for generating random conjugates (where conjugating word has varying length between 1 and 10, say) of relators or their inverses, picking a random position in the representative of the trivial group element that’s already been created, and inserting the conjugate at that position


**TESTING**

In [26]:
#symmetric group S5
sym5 = np.array([[1,3,2,2],[3,1,3,2],[2,3,1,3],[2,2,3,1]])

#symmetric group S4
sym4 = np.array([[1,3,2],[3,1,3],[2,3,1]])

#symmetric group S3
sym3 = np.array([[1,3], [3,1]]) ###EDGE CASE HANDLING

#example with entries at infinity
m = np.array([[1,2,3,2],[2,1,4,2],[3,4,1,np.inf],[2,2,np.inf,1]])

### Preliminary Functions

In [27]:
def cox_gen(matrix):
  n = np.sum(matrix == 1)            #masks for 1s and sums all true values
  generators = list(range(1,n+1))    #generates integers from 1 through n
  return generators


def cox_rel(matrix):
  #generate pairs to check cases where s != s'
  generators = cox_gen(matrix)
  pairs = [[generators[i], generators[j]] for i in range(len(generators)) for j in range(i + 1, len(generators))]
  relators = []

  #getting relators of form s^2
  for g in generators:
    relators.append([g,g])

  #getting braid relators
  for p in pairs:
    m = matrix[p[0]-1,p[1]-1]       #subtracting 1 allows us to retrieve the correct row and column, eg what we call row 1 is actually indexed as row 0
    if np.any(np.isinf(m)):         #skipping to the next pair if m(s,s') = infinity
      continue
    relators.append(p*(int(m)))     #appends the pair p m times, representing the relation (ss')^m(s,s') = e
  return relators

def artin_gen(matrix):
  n = np.sum(matrix == 1)            #masks for 1s and sums all true values
  generators = list(range(-n,n+1))   #generates integers from -n through n
  generators.remove(0)
  return generators

def artin_rel(matrix):
  #generate pairs to check cases where s != s'
  generators = cox_gen(matrix)
  pairs = [(generators[i], generators[j]) for i in range(len(generators)) for j in range(i + 1, len(generators))]

  relators = []

  #retrieving length m from m(s,s')
  for p in pairs:
    m = matrix[p[0]-1,p[1]-1]
    if np.any(np.isinf(m)):         #skipping to the next pair if m(s,s') = infinity
      continue

    #building pi(s,s',m)
    pi = []

    #alternating between s and s' for an m-length list
    for i in range(int(m)):
      if i % 2 == 0:
        pi.append(p[0])     #even indices give s
      else:
        pi.append(p[1])     #odd indices give s'

    #building pi(s',s, m) inverse
    pi_inv = []
    for i in range(int(m)):               #same process as above except
      if i % 2 != 0:
        pi_inv.append(p[0])               #even indices now give s'
      else:
        pi_inv.append(p[1])               #and odd indices give s
    pi_inv = list(map(neg, pi_inv))       #flip signs to denote inverses

    #combining pi and pi inverse
    relators.append(pi + pi_inv)

  return relators

# Coxeter Version of Subroutine B

In [28]:
from re import sub

# Note: If there are only 2 or less generators present in the finite relators of a Coxeter Group the following function won't work
##### If there are 2 generators present in the finite relators, the only valid trivial word is alternating between the generators (ie s1s2s1s2s1s2 repeating or s2s1s2s1s2s1 repeating)
##### If there is only 1 generator present in the finite relators, abandon the activity as it is impossible to generate visually unreducable trivial words

def subroutine_b_cox(t, set_of_generators, set_of_relators):
  t = reduce_coxeter_word(t)
  size_of_trivial = len(t)
  insertion_point = random.randint(0, size_of_trivial)

  #letters between insertion point
  a = None
  b = None
  if size_of_trivial > 0:
    if insertion_point > 0:
        a = t[insertion_point - 1]
    if insertion_point < size_of_trivial:
        b = t[insertion_point]

  ####### randomly generate word w
  w_length = random.randint(1,10)
  w = []
  for i in range(w_length):
    w.append(random.choice(set_of_generators))

  # reduce w using subroutine A
  w = reduce_coxeter_word(w)

  if len(w) == 0:
    return subroutine_b_cox(t, set_of_generators, set_of_relators)

  # calculate w inverse(note its for coxeter group)
  w_inv = w[::-1]

  # make sure word w does not create visually reducable words
  if (a is not None and a == w[0]) or (b is not None and w_inv[-1] == b):
    return subroutine_b_cox(t, set_of_generators, set_of_relators)



####### pick a relator that is not visually reducable
####### Choose a relator that avoids obvious cancellation with w and w_inv
  non_squares = [rel for rel in set_of_relators if rel[0] != rel[1]]
  candidates = []

  for rel in non_squares:
      for rel_tuple in [rel, rel[::-1]]:
          if w[-1] != rel_tuple[0] and rel_tuple[-1] != w_inv[0]:
              candidates.append(rel_tuple)

  if not candidates:
      return subroutine_b_cox(t, set_of_generators, set_of_relators)

  r = random.choice(candidates)

  ####### Insert conjugate: w + r + w_inv
  conjugate = w + list(r) + w_inv
  t[insertion_point:insertion_point] = conjugate
  return t



## **Coxeter sub_b Trial Run**

In [29]:
t=[1,1]
generators = cox_gen(m)
relators = cox_rel(m)
print(f"generators:",generators)
print(f"relators:",relators)
print(f"trivial string:",subroutine_b_cox(t, generators, relators))


generators: [1, 2, 3, 4]
relators: [[1, 1], [2, 2], [3, 3], [4, 4], [1, 2, 1, 2], [1, 3, 1, 3, 1, 3], [1, 4, 1, 4], [2, 3, 2, 3, 2, 3, 2, 3], [2, 4, 2, 4]]
trivial string: [2, 3, 2, 1, 2, 3, 1, 3, 1, 3, 1, 2, 1, 2, 3, 2]


# Artin Version of Subroutine B

In [30]:
# Note: good to run unless only 1 generator. Then, abandon.
def subroutine_b_artin(t, set_of_generators, set_of_relators):
    t = reduce_artin_word(t)
    size_of_trivial = len(t)
    insertion_point = random.randint(0, size_of_trivial)

    # Get neighbors to avoid cancellation at insertion boundaries
    a = t[insertion_point - 1] if insertion_point > 0 else None
    b = None
    if len(t) > 0:
      b = t[insertion_point] if insertion_point < size_of_trivial else None

    ####### Generate random reduced word w
    w_length = random.randint(1, 10)
    w = [random.choice(set_of_generators) for _ in range(w_length)]
    w = reduce_artin_word(w)
    if not w:
        return subroutine_b_artin(t, set_of_generators, set_of_relators)

    w_inv = [-g for g in reversed(w)]

    # Early check: avoid reduction at boundaries with t
    if (a is not None and w and a == -w[0]) or (b is not None and w_inv and w_inv[-1] == -b):
        return subroutine_b_artin(t, set_of_generators, set_of_relators)


    ####### Choose a non-reducing relator
    valid_relators = []
    for rel in set_of_relators:
        # Skip trivial relators like (g, -g)
        if len(rel) == 2 and rel[0] == -rel[1]:
            continue
        for rel_tuple in [list(rel), [-g for g in reversed(rel)]]:
            if w and rel_tuple and w[-1] == -rel_tuple[0]:
                continue  # would cancel with end of w
            if rel_tuple and w_inv and rel_tuple[-1] == -w_inv[0]:
                continue  # would cancel with start of w_inv
            rel_reduced = reduce_artin_word(rel_tuple)
            if not rel_reduced:
              continue
            valid_relators.append(rel_tuple)

    if not valid_relators:
        return subroutine_b_artin(t, set_of_generators, set_of_relators)

    r = random.choice(valid_relators)

    ####### Form conjugate and insert
    conjugate = w + r + w_inv
    t[insertion_point:insertion_point] = conjugate
    return t

### **Artin Group sub_B Trial Run**

In [31]:
t=[1,-1]
generators = artin_gen(sym4)
relators = artin_rel(sym4)
print(f"generators:",generators)
print(f"relators:",relators)
print(f"trivial string:",subroutine_b_artin(t, generators, relators))

generators: [-3, -2, -1, 1, 2, 3]
relators: [[1, 2, 1, -2, -1, -2], [1, 3, -3, -1], [2, 3, 2, -3, -2, -3]]
trivial string: [3, 1, -3, -1, 3, 1, 2, 1, -2, -1, -2, -3, 1, 3, -1, -3]
